<a href="https://colab.research.google.com/github/37stu37/Hikurangi_mhzrd/blob/master/mhzrd_hikurangi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import os
import random

In [0]:
# raw data
earthquake_csv = pd.read_excel('/content/drive/My Drive/04_Cloud/01_Work/GNS/SSIF_2019/6 Flagship_Hikurangi/tsunami_bathtub_model/SubductionSources_27June2019.xls')
tsunami_csv = pd.read_csv('/content/drive/My Drive/04_Cloud/01_Work/GNS/SSIF_2019/6 Flagship_Hikurangi/tsunami_bathtub_model/BTable_20km_Hpct99_27June2019.csv')
# node_csv = pd.read_csv('/content/drive/My Drive/04_Cloud/01_Work/GNS/SSIF_2019/6 Flagship_Hikurangi/node.csv')
earthquake_sources = earthquake_csv.UID.to_list()

In [4]:
earthquake_csv.head(1)

,UID,Subduction Zone,Segment,Plate pairs,Left_E_LONG,Left_N_LAT,Left_REL_VEL,Left_REL_AZI,Right_E_LONG,Right_N_LAT,Right_REL_VEL,Right_REL_AZI,Length (km),Dip - pref,Dip direction,Trench depth,Updip_pref,Updip_min,Updip_max,Down-dip depth - pref,Down-dip depth - min,Down-dip depth - max,Width - pref,Width - min,Width - max,Coupling coefficient - pref,Coupling coefficient - min,Coupling coefficient - max,"SED (whole seg, m)",SED sigma,Mmax - pref,Mmax - min,Mmax - max,B-value - pref,B-value - min,B-value - max
0,AK_D_Alaska_Whole,Alaska/Aleutians,Whole margin,PA\NA,164.066,55.209,74.6,311,-140.128,60.381,49.0,350,4130,14,NaN,6.0,12.0,6.0,24.0,40,26,48,122,30,192,0.55,0.42,0.77,NaN,NaN,9.4,9.2,9.6,0.93,0.67,1.2


In [0]:
### test sampling random earthquake
# for i in range(1):
#   e = random.choice(earthquake_sources)
#   print(type(e))



---

Functions


---




In [0]:
class Earthquake:

    def __init__(self, scenario_number, earthquake_csv=earthquake_csv):
        # select a source area to trigger a rupture
        e = random.choice(earthquake_sources)
        # crop dataframe to the selected source
        earthquake_df = earthquake_csv[['UID', 'B-value - pref', 'a-value']] # ! add "a" value later on
        earthquake_df = earthquake_df[earthquake_df.UID = e]
        # create the discrete frequency / magnitude distribution from the source selected
        magnitudes = np.arange(0, 11, 0.1)
        frequencies = 10**(earthquake_df['a_value']-(earthquake_df['B-value - pref']*magnitudes))
        frequencies = frequencies / frequencies.sum() # make the probability of having an earthquake from 0 to 1
        # select a random magnitude from the distribution 
        magnitude_RNG = np.random.choice(magnitudes, 1, p=frequencies)
        # (back)find the recurrence time generated
        magnitudes = magnitudes.to_list
        index = magnitudes.index(magnitudes_RNG[0])
        rec_time = 1/frequencies[index] # get the recurrence years - easier to np,interp if distribution is monotonicaly increasing
        # return the specifics of the earthquake source
        self.uid = earthquake_df.UID
        self.magnitude = magnitude_RNG[0]
        self.rec_time = rec_time[0]
        self.scenario_number = scenario_number

class EarthquakeGroundMotions(Earthquake):
    def __init__(self, scenario_number: int, earthquake_csv=earthquake_csv, node_file=node_file_csv):
      super(EarthquakeGroundMotions, self).__init__(scenario_number, earthquake_csv=earthquake_csv)
      self.shaking_file = self.shake(scenario_number)

    def Shaking(self):
        # access the node file
        nodes = pd.read_csv(self.node_file)
        if self.uid.str.contains("Hik")]:
          recurrence_times = [1, 10, 100, 1000]
          shaking_columns = ['PGA_1y', 'PGA_10y', 'PGA_100y', 'PGA_1000y']
          # create a shaking value at each node based on the event recurrence time. faster solution here https://stackoverflow.com/questions/43765796/interpolating-values-from-a-dataframe-based-on-a-column-value
          nodes['shaking'] = nodes.apply(lambda x: np.interp(self.rec_time, recurrence_times, [x.PGA_1y, x.PGA_10y, x.PGA_100y, x.PGA_1000y]), axis=1)
        else :
          nodes['shaking'] = 0
        return nodes[['nodeID', 'shaking']]


In [0]:
class Tsunami:

    def __init__(self, eq_uid, eq_magnitude, Bij):
        self.eq_uid = earthquake_df.UID
        self.magnitude = magnitude_RNG[0]
        self.rec_time = rec_time[0]
    # calculate Ht for all target areas (Bij) using the Earthquake Mw Ht = 10^Mw-Bij
    # from the distance to shore value, calculate the Water depth Wd = (Ht*2) - (distance from shore in meters/400)

In [0]:
class Landslide(EarthquakeGroundMotions,Rainfall):
    def __init__(self):
        super(Landslide, self).__init__()
        shaking_df = self.get_shaking()

    def get_shaking(self):
        return pd.read_csv(self.shaking_file)
    # get the shaking at node location
    # get the landslide suscpetibility
    # sample from the area / volume relationship
    # get a volume and runout + target node

1) random selection of the source earthquake location & sampling of the probability / intensity distribution (N = 10^(a-bMw))

2) generation of tsunami based on the Bij value (Ht = 10^(Mw-Bij) & calculation of depth of inundation (Htx2 - decay)